# Exercise 3: ETL Pipeline for E-commerce Analytics

In [1]:
import json
import pandas as pd

json_file = json.load(open('sample-dataset-3.json'))
json_file

[{'transaction_id': 'T123456',
  'timestamp': '2024-01-15T14:30:00',
  'customer': {'id': 'CUS123',
   'region': 'North',
   'segment': 'Premium',
   'join_date': '2023-01-01'},
  'items': [{'product_id': 'P789',
    'category': 'Electronics',
    'subcategory': 'Smartphones',
    'price': 499.99,
    'quantity': 1,
    'discount': 0.0},
   {'product_id': 'P456',
    'category': 'Accessories',
    'subcategory': 'Phone Cases',
    'price': 29.99,
    'quantity': 2,
    'discount': 5.0}],
  'payment_method': 'credit_card',
  'status': 'completed',
  'shipping': {'method': 'express',
   'cost': 15.99,
   'address': {'city': 'Boston', 'state': 'MA', 'country': 'USA'}}},
 {'transaction_id': 'T123457',
  'timestamp': '2024-01-15T14:45:00',
  'customer': {'id': 'CUS456',
   'region': 'South',
   'segment': 'Standard',
   'join_date': '2023-06-15'},
  'items': [{'product_id': 'P234',
    'category': 'Books',
    'subcategory': 'Fiction',
    'price': 24.99,
    'quantity': 3,
    'discount': 

# 1.Create a function that transforms raw transaction data:

### Flatten nested JSON structures using lambda functions

In [2]:
flattened_transaction = lambda t : [
{
**{
        'transaction_id' : t['transaction_id'],
        'timestamp' : t['timestamp'],
        'customer_id' : t['customer']['id'],
        'customer_region' : t['customer']['region'],
        'customer_segment' : t['customer']['segment'],
        'customer_joindate' : t['customer']['join_date'],
        'payment_method': t['payment_method'],
        'status': t['status'],
        'shipping_method': t['shipping']['method'],
        'shipping_cost' : t['shipping']['cost'],
        'city' : t['shipping']['address']['city'],
        'state' : t['shipping']['address']['state'],
        'country' : t['shipping']['address']['country']
    },
    **{
        'product_id' : item['product_id'],
        'category' : item['category'],
        'subcategory' : item['subcategory'],
        'price' : item['price'],
        'quantity' : item['quantity'],
        'discount' : item['discount']
    },
 }
 for item in t['items']  
]


In [3]:
flattened_data = sum(list(map(flattened_transaction,json_file)), []) # Applying lambda function created for flattening complex json datas using map
flattened_data

[{'transaction_id': 'T123456',
  'timestamp': '2024-01-15T14:30:00',
  'customer_id': 'CUS123',
  'customer_region': 'North',
  'customer_segment': 'Premium',
  'customer_joindate': '2023-01-01',
  'payment_method': 'credit_card',
  'status': 'completed',
  'shipping_method': 'express',
  'shipping_cost': 15.99,
  'city': 'Boston',
  'state': 'MA',
  'country': 'USA',
  'product_id': 'P789',
  'category': 'Electronics',
  'subcategory': 'Smartphones',
  'price': 499.99,
  'quantity': 1,
  'discount': 0.0},
 {'transaction_id': 'T123456',
  'timestamp': '2024-01-15T14:30:00',
  'customer_id': 'CUS123',
  'customer_region': 'North',
  'customer_segment': 'Premium',
  'customer_joindate': '2023-01-01',
  'payment_method': 'credit_card',
  'status': 'completed',
  'shipping_method': 'express',
  'shipping_cost': 15.99,
  'city': 'Boston',
  'state': 'MA',
  'country': 'USA',
  'product_id': 'P456',
  'category': 'Accessories',
  'subcategory': 'Phone Cases',
  'price': 29.99,
  'quantity': 

In [4]:
df = pd.DataFrame(flattened_data) # flattened data converted to dataframe
df.head()

,transaction_id,timestamp,customer_id,customer_region,customer_segment,customer_joindate,payment_method,status,shipping_method,shipping_cost,city,state,country,product_id,category,subcategory,price,quantity,discount
0,T123456,2024-01-15T14:30:00,CUS123,North,Premium,2023-01-01,credit_card,completed,express,15.99,Boston,MA,USA,P789,Electronics,Smartphones,499.99,1,0.0
1,T123456,2024-01-15T14:30:00,CUS123,North,Premium,2023-01-01,credit_card,completed,express,15.99,Boston,MA,USA,P456,Accessories,Phone Cases,29.99,2,5.0
2,T123457,2024-01-15T14:45:00,CUS456,South,Standard,2023-06-15,paypal,completed,standard,5.99,Miami,FL,USA,P234,Books,Fiction,24.99,3,10.0
3,T123458,2024-01-15T15:00:00,CUS789,West,Premium,2022-03-20,credit_card,pending,express,25.99,Los Angeles,CA,USA,P789,Electronics,Smartphones,499.99,1,50.0
4,T123458,2024-01-15T15:00:00,CUS789,West,Premium,2022-03-20,credit_card,pending,express,25.99,Los Angeles,CA,USA,P555,Electronics,Tablets,299.99,1,0.0


### Calculate total transaction values

In [5]:
print (f" Total Transaction Values : {((df['price']*df['quantity']) - df['discount']).sum()}")

 Total Transaction Values : 2049.86


###  Extract unique product categories using map() and set()

In [6]:
set(df['category'])

{'Accessories', 'Books', 'Clothing', 'Electronics', 'Home'}

#  2. Create analysis functions that:

### Group transactions by region and calculate regional sales

In [7]:
df['revenue'] = (df['price']*df['quantity']) - df['discount']
df.groupby('customer_region')['revenue'].sum().reset_index()

,customer_region,revenue
0,East,124.97
1,North,734.96
2,South,64.97
3,West,1124.96


### Find top-selling products using sorted() with custom key

In [8]:
top_selling_products = sorted(flattened_data, key= lambda x: x['quantity'], reverse=True)

for products in top_selling_products:
    print(f"Product ID : {products['product_id']} , Quantity Sold {products['quantity']}")

Product ID : P234 , Quantity Sold 3
Product ID : P456 , Quantity Sold 2
Product ID : P777 , Quantity Sold 2
Product ID : P333 , Quantity Sold 2
Product ID : P789 , Quantity Sold 1
Product ID : P789 , Quantity Sold 1
Product ID : P555 , Quantity Sold 1
Product ID : P444 , Quantity Sold 1
Product ID : P888 , Quantity Sold 1


### Calculate average transaction value by payment method

In [9]:
df.groupby('payment_method').agg(average_transaction_value=('revenue','mean')).reset_index()

,payment_method,average_transaction_value
0,credit_card,335.986
1,debit_card,62.485
2,gift_card,179.990
3,paypal,64.970


# 3. Create a report generation function that:
- ### Filters completed transactions using filter()
- ### Sorts data by multiple criteria using lambda
- ### Generates summary statistics for different time periods

In [10]:
def report_generation(df,columns=['price','quantity'],asc_or_desc=[False,False]):
    filtered_df = df[df['status']=='completed']
    multiple_columns_sorted = df.sort_values(by=columns, ascending=asc_or_desc)
    df['timestamp'] = pd.to_datetime(df['timestamp'])   
    different_time_period = df.groupby('timestamp').agg(customer_count=('customer_id','count'), revenue=('revenue','sum')).reset_index()
    return filtered_df,multiple_columns_sorted,different_time_period

In [11]:
filtered_df,multiple_columns_sorted,different_time_period = report_generation(df)

In [12]:
filtered_df

,transaction_id,timestamp,customer_id,customer_region,customer_segment,customer_joindate,payment_method,status,shipping_method,shipping_cost,city,state,country,product_id,category,subcategory,price,quantity,discount,revenue
0,T123456,2024-01-15T14:30:00,CUS123,North,Premium,2023-01-01,credit_card,completed,express,15.99,Boston,MA,USA,P789,Electronics,Smartphones,499.99,1,0.0,499.99
1,T123456,2024-01-15T14:30:00,CUS123,North,Premium,2023-01-01,credit_card,completed,express,15.99,Boston,MA,USA,P456,Accessories,Phone Cases,29.99,2,5.0,54.98
2,T123457,2024-01-15T14:45:00,CUS456,South,Standard,2023-06-15,paypal,completed,standard,5.99,Miami,FL,USA,P234,Books,Fiction,24.99,3,10.0,64.97
6,T123459,2024-01-15T15:15:00,CUS101,East,Standard,2024-01-01,debit_card,completed,standard,7.99,New York,NY,USA,P333,Clothing,Shirts,39.99,2,15.0,64.98
7,T123459,2024-01-15T15:15:00,CUS101,East,Standard,2024-01-01,debit_card,completed,standard,7.99,New York,NY,USA,P444,Clothing,Pants,59.99,1,0.0,59.99


In [13]:
multiple_columns_sorted

,transaction_id,timestamp,customer_id,customer_region,customer_segment,customer_joindate,payment_method,status,shipping_method,shipping_cost,city,state,country,product_id,category,subcategory,price,quantity,discount,revenue
0,T123456,2024-01-15T14:30:00,CUS123,North,Premium,2023-01-01,credit_card,completed,express,15.99,Boston,MA,USA,P789,Electronics,Smartphones,499.99,1,0.0,499.99
3,T123458,2024-01-15T15:00:00,CUS789,West,Premium,2022-03-20,credit_card,pending,express,25.99,Los Angeles,CA,USA,P789,Electronics,Smartphones,499.99,1,50.0,449.99
4,T123458,2024-01-15T15:00:00,CUS789,West,Premium,2022-03-20,credit_card,pending,express,25.99,Los Angeles,CA,USA,P555,Electronics,Tablets,299.99,1,0.0,299.99
5,T123458,2024-01-15T15:00:00,CUS789,West,Premium,2022-03-20,credit_card,pending,express,25.99,Los Angeles,CA,USA,P777,Electronics,Headphones,199.99,2,25.0,374.98
8,T123460,2024-01-15T15:30:00,CUS202,North,Premium,2023-08-15,gift_card,cancelled,standard,9.99,Chicago,IL,USA,P888,Home,Kitchen,199.99,1,20.0,179.99
7,T123459,2024-01-15T15:15:00,CUS101,East,Standard,2024-01-01,debit_card,completed,standard,7.99,New York,NY,USA,P444,Clothing,Pants,59.99,1,0.0,59.99
6,T123459,2024-01-15T15:15:00,CUS101,East,Standard,2024-01-01,debit_card,completed,standard,7.99,New York,NY,USA,P333,Clothing,Shirts,39.99,2,15.0,64.98
1,T123456,2024-01-15T14:30:00,CUS123,North,Premium,2023-01-01,credit_card,completed,express,15.99,Boston,MA,USA,P456,Accessories,Phone Cases,29.99,2,5.0,54.98
2,T123457,2024-01-15T14:45:00,CUS456,South,Standard,2023-06-15,paypal,completed,standard,5.99,Miami,FL,USA,P234,Books,Fiction,24.99,3,10.0,64.97


In [14]:
different_time_period

,timestamp,customer_count,revenue
0,2024-01-15 14:30:00,2,554.97
1,2024-01-15 14:45:00,1,64.97
2,2024-01-15 15:00:00,3,1124.96
3,2024-01-15 15:15:00,2,124.97
4,2024-01-15 15:30:00,1,179.99
